In [1]:
import pandas as pd
import re
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.4 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import gdown

# Link yang diperoleh dari Google Drive
file_id = '1sKLR2K-VVcmOasYRU03bOAJPDBNJ24lk'  # Ganti dengan file ID kamu
url = f'https://drive.google.com/uc?id={file_id}'

# Unduh file
output = 'ulasan_aplikasi_traveloka3.csv'
gdown.download(url, output, quiet=False)

# Baca file CSV
df = pd.read_csv(output)
df.head()


Downloading...
From: https://drive.google.com/uc?id=1sKLR2K-VVcmOasYRU03bOAJPDBNJ24lk
To: /content/ulasan_aplikasi_traveloka3.csv
100%|██████████| 7.20M/7.20M [00:00<00:00, 165MB/s]


,content
0,"Tiap pesan hotel data gak jelas, map ngaco, be..."
1,Saya kecewa dengan Aplikasi traveloka yg sekar...
2,"Harga yg tertera berubah ubah setiap detik, ti..."
3,"Aplikasi sampah, saya barusan melakukan pesana..."
4,Sementara 1 dulu klo sudah baik fitur kamerany...


In [3]:
df.dropna(inplace=True)
clean_df = df
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158173 entries, 0 to 158194
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   content  158173 non-null  object
dtypes: object(1)
memory usage: 2.4+ MB


In [4]:
# preprocesing text
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in text]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [5]:
import requests

# Fungsi untuk mengunduh dan memproses slang words
def download_slangwords(url):
    response = requests.get(url)
    response.raise_for_status()  # Akan menimbulkan exception jika ada error dalam permintaan
    lines = response.text.splitlines()
    slang_dict = {}
    for line in lines:
        # Asumsikan format file adalah 'slang_word replacement_word'
        parts = line.split()
        if len(parts) == 2:
            slang_dict[parts[0].lower()] = parts[1]
    return slang_dict

# URL dari file slang words
url = 'https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt'

# Mengunduh dan memproses slang words
slangwords = download_slangwords(url)

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [6]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

clean_df

,content,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,"Tiap pesan hotel data gak jelas, map ngaco, be...",Tiap pesan hotel data gak jelas map ngaco bebe...,tiap pesan hotel data gak jelas map ngaco bebe...,tiap pesan hotel data gak jelas map ngaco bebe...,"[tiap, pesan, hotel, data, gak, jelas, map, ng...","[pesan, hotel, data, map, ngaco, kali, nyasar,...",pesan hotel data map ngaco kali nyasar letih c...
1,Saya kecewa dengan Aplikasi traveloka yg sekar...,Saya kecewa dengan Aplikasi traveloka yg sekar...,saya kecewa dengan aplikasi traveloka yg sekar...,saya kecewa dengan aplikasi traveloka yg sekar...,"[saya, kecewa, dengan, aplikasi, traveloka, yg...","[kecewa, aplikasi, traveloka, yg, nipu, seharu...",kecewa aplikasi traveloka yg nipu seharus kalo...
2,"Harga yg tertera berubah ubah setiap detik, ti...",Harga yg tertera berubah ubah setiap detik tid...,harga yg tertera berubah ubah setiap detik tid...,harga yg tertera berubah ubah setiap detik tid...,"[harga, yg, tertera, berubah, ubah, setiap, de...","[harga, yg, tertera, berubah, ubah, detik, men...",harga yg tertera berubah ubah detik menampilka...
3,"Aplikasi sampah, saya barusan melakukan pesana...",Aplikasi sampah saya barusan melakukan pesanan...,aplikasi sampah saya barusan melakukan pesanan...,aplikasi sampah saya barusan melakukan pesanan...,"[aplikasi, sampah, saya, barusan, melakukan, p...","[aplikasi, sampah, barusan, pesanan, hotel, tr...",aplikasi sampah barusan pesanan hotel travelok...
4,Sementara 1 dulu klo sudah baik fitur kamerany...,Sementara dulu klo sudah baik fitur kameranya...,sementara dulu klo sudah baik fitur kameranya...,sementara dulu klo sudah baik fitur kameranya ...,"[sementara, dulu, klo, sudah, baik, fitur, kam...","[klo, fitur, kameranya, ambil, foto, dokumen, ...",klo fitur kameranya ambil foto dokumen kayak k...
...,...,...,...,...,...,...,...
158190,👻,,,,[],[],
158191,😈😈😈😈😈😈😈😈,,,,[],[],
158192,👍👍👍👍👍👍👍👍👍,,,,[],[],
158193,😊😊,,,,[],[],


In [7]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [8]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if score > 0:
        polarity = 'positive'
        # Jika skor sentimen lebih besar dari 0, maka polaritas adalah positif
    elif score < 0:
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif
    else:
        polarity = 'neutral'
    # Jika skor sentimen tepat 0, maka polaritas adalah netral

    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [9]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    75538
neutral     46840
negative    35795
Name: count, dtype: int64


**Pelatihan: LSTM, Ekstraksi Fitur: Word2Vec, Pembagian Data: 80/20**

In [10]:
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

# Ambil data teks
texts = clean_df['text_akhir'].tolist()

def tokenize_text(text):
    return text.split()  # Mengembalikan daftar token (kata)

# Tokenisasi teks
tokenized_texts = [tokenize_text(text) for text in clean_df['text_akhir']]


# Latih model Word2Vec
model = Word2Vec(vector_size=300, window=5, min_count=5, workers=4)

# Bangun vocabulary
model.build_vocab(tokenized_texts)

print(f"Vocabulary size: {len(model.wv)}")

# Latih model
model.train(tokenized_texts, total_examples=model.corpus_count, epochs=10)

# Simpan model
model.save("word2vec_model.model")

Vocabulary size: 6870


In [11]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = Word2Vec.load("word2vec_model.model")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_df['text_akhir'])
sequences = tokenizer.texts_to_sequences(clean_df['text_akhir'])
word_index = tokenizer.word_index

# Buat matriks embedding
vocab_size = len(word_index) + 1
embedding_dim = model.vector_size  # Sesuaikan dengan dimensi model Word2Vec
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]

print("Embedding matrix sample:")
print(embedding_matrix[:5])

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X_sequences = tokenizer.texts_to_sequences(clean_df['text_akhir'])
max_length = max(len(seq) for seq in X_sequences)
X_padded = pad_sequences(X_sequences, maxlen=max_length)

label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(clean_df['polarity'])

# Memecah data
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

Embedding matrix sample:
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.09910809 -0.46286103  0.42308518 ...  0.14739712 -0.25148785
   0.32811961]
 [-0.05166174  0.36926174 -0.0097675  ...  0.23271313  0.60036469
  -0.87465835]
 [ 0.94600433  0.11082868 -0.92836529 ...  0.27864888 -0.01239061
   0.04694785]
 [-0.14976525  0.16412513 -0.25830743 ...  0.17413847  0.17518522
   0.11476094]]


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Latih model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 75s 33ms/step - accuracy: 0.8161 - loss: 0.4806 - val_accuracy: 0.9097 - val_loss: 0.2432
Epoch 2/10
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 65s 33ms/step - accuracy: 0.9046 - loss: 0.2823 - val_accuracy: 0.9187 - val_loss: 0.2244
Epoch 3/10
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 81s 32ms/step - accuracy: 0.9169 - loss: 0.2473 - val_accuracy: 0.9255 - val_loss: 0.2080
Epoch 4/10
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 83s 33ms/step - accuracy: 0.9246 - loss: 0.2250 - val_accuracy: 0.9306 - val_loss: 0.1970
Epoch 5/10
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 83s 33ms/step - accuracy: 0.9287 - loss: 0.2149 - val_accuracy: 0.9317 - val_loss: 0.1866
Epoch 6/10
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 82s 33ms/step - accuracy: 0.9343 - loss: 0.2001 - val_accuracy: 0.9347 - val_loss: 0.1842
Epoch 7/10
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 82s 33ms/step - accuracy: 0.9365 - loss: 0.1908 - val_accuracy: 0.9372 - val_loss: 0.1773
Epoch 8/10
1978/1978 ━━━━━━━━━━━━━━━━━━━━ 82s 33ms/step - accuracy: 0.9400 -

In [13]:
# Evaluasi model pada data training dan testing
loss_train, accuracy_train = model.evaluate(X_train, y_train)
print(f"Training Accuracy: {accuracy_train * 100:.2f}%")

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Testing Accuracy: {accuracy * 100:.2f}%")

3955/3955 ━━━━━━━━━━━━━━━━━━━━ 47s 12ms/step - accuracy: 0.9515 - loss: 0.1367
Training Accuracy: 95.12%
989/989 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.9404 - loss: 0.1733
Testing Accuracy: 94.04%
